In [16]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import os
import tabula
import numpy as np
import re
import requests

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\\delete\\US EPA'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [6]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\US EPA","plugins.always_open_pdf_externally": True}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.epa.gov/pesticide-worker-safety/restricted-use-products-rup-report")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="main"]/div/div[1]/div[2]/div[1]/article/div[2]/div/ul[3]/li/span/a').click()
time.sleep(10)
driver.close()

In [7]:
file_path = "Y:/delete/US EPA/rup-report-10-08-2021.pdf"
df = tabula.read_pdf(file_path, pages='all')
df1 = pd.concat(df)
df1.head(2)

,Registration #,Product Name,Company\rNumber,Company Name,Active Ingredient(s),% Active\rIngredient(s)
0,4-152,BONIDE ORCHARD MOUSE BAIT,4,"BONIDE PRODUCTS, LLC",Zinc phosphide (Zn3P2),2
1,100-1075,FORCE 3G INSECTICIDE,100,"SYNGENTA CROP PROTECTION, LLC",Tefluthrin,3


In [20]:
mylist = df1['Active Ingredient(s)'].to_numpy()
mylist = np.unique(mylist)

In [21]:
cas = np.array([])
x=0
p = re.compile("[0-9]+-[0-9]+-[0-9]+")
l2 = []
for i in mylist:
    print(i)
    base_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'+str(i)+'/synonyms/txt'
    r = requests.get(base_url)
    data = r.text
    lst = data.split('\n')
    if lst[0] != 'Status: 404' :
        l2 = [ s for s in lst if p.match(s) ]
        if len(l2)>0 :
                    l2 = np.array([l2[0]])        
        else:
            l2 = ['NA']
    else:
        if i.find('(') > 1:
            x = i.split(' (')
            base_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'+str(x[0].strip())+'/synonyms/txt'
            r = requests.get(base_url)
            data = r.text 
            lst = data.split('\n')
            if lst[0] != 'Status: 404' :
                l2 = [ s for s in lst if p.match(s) ]
                if len(l2)>0 :
                    l2 = np.array([l2[0]])        
                else:
                    l2 = ['NA']
            else:
                l2 = ['NA']
        else:
                l2 = ['NA']
    print(l2)            
    cas = np.concatenate((cas, l2))


1-Decanaminium, N-decyl-N, N-dimethyl-, chloride
['NA']
2, 4-D, 2-ethylhexyl ester, Atrazine
['NA']
2, 4-D, Choline salt, Picloram
['NA']
2, 4-D, isopropyl ester
['NA']
salt-D, triisopropanolamine salt, Picloram, triisopropanolamine
['NA']
4-Aminopyridine
['504-24-5']
Abamectin
['NA']
Abamectin, Bifenthrin
['NA']
Abamectin, Cyantraniliprole
['NA']
Abamectin, Thiamethoxam
['NA']
Azoxystrobin, Thiabendazoleudioxonil, Metalaxyl-M,
['NA']
Acephate, Bifenthrin
['NA']
Acetamiprid
['135410-20-7']
Acetamiprid, Bifenthrin
['NA']
Acetochlor
['34256-82-1']
Acetochlor, Atrazine
['NA']
Acetochlor, Carbamothioic acid, dipropyl-, S-ethyl ester
['NA']
Acrolein
['107-02-8']
Afidopyropen, Alpha-cypermethrin
['NA']
Aldicarb
['116-06-3']
Alpha-cypermethrin
['52315-07-8']
Aluminum phosphide
['20859-73-8']
Aminopyralid-potassium, Picloram-potassium, Fluroxypyr-meptyl
['NA']
Arsenic acid
['7778-39-4']
Arsenic oxide  (As2O5), Chromic acid, Cupric oxide
['NA']
Arsenic oxide  (As2O5), Zinc oxide, Cupric oxide
[

In [22]:
dataset = pd.DataFrame({'mylist':mylist,'CAS Number':cas},columns=['mylist','CAS Number'])
dataset = dataset.loc[dataset['CAS Number'] != 'NA']
dataset.dropna(inplace = True)
dataset

,mylist,CAS Number
5,4-Aminopyridine,504-24-5
12,Acetamiprid,135410-20-7
14,Acetochlor,34256-82-1
17,Acrolein,107-02-8
19,Aldicarb,116-06-3
...,...,...
209,Zinc phosphide (Zn3P2),1314-84-7
210,alpha-Chlorohydrin,96-24-2
211,beta-Cyfluthrin,1820573-27-0
213,gamma-Cyhalothrin,76703-62-3


In [24]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not in list'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)
result = pd.merge(dataset, CAS, how="outer", on="CAS Number")

,mylist,CAS Number,value
0,4-Aminopyridine,504-24-5,NaN
1,Acetamiprid,135410-20-7,NaN
2,Acetochlor,34256-82-1,NaN
3,Acrolein,107-02-8,NaN
4,Aldicarb,116-06-3,NaN
...,...,...,...
87,NaN,14807-96-6,Not in list
88,NaN,7631-86-91,Not in list
89,NaN,9002-84-0,Not in list
90,NaN,65-85-0,Not in list


In [25]:
result['Jurisdiction'] = 'US'
result['RegulatoryAgency'] = 'Environment Protection Aganecy (EPA)'
result['TypeOfRegulation'] = 'Use'
result['Criteria'] = 'Pesticide use restrictions'

result['Description'] = 'Restricted'
result.loc[result['value'] == 'Not in list', 'Description'] = 'Not Restricted'

result['HazardClassification'] = 'High'
result.loc[result['value'] == 'Not in list', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'On Restricted Use Products (RUP) list'
result.loc[result['value'] == 'Not in list', 'CauseOfClassification'] = 'Not on Restricted Use Products (RUP) list'

result['SourceName'] = 'US EPA'
result['SourceURL'] = 'https://www.epa.gov/pesticide-worker-safety/restricted-use-products-rup-report'

result.head(2)

,mylist,CAS Number,value,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,4-Aminopyridine,504-24-5,NaN,US,Environment Protection Aganecy (EPA),Use,Pesticide use restrictions,Restricted,High,On Restricted Use Products (RUP) list,US EPA,https://www.epa.gov/pesticide-worker-safety/re...
1,Acetamiprid,135410-20-7,NaN,US,Environment Protection Aganecy (EPA),Use,Pesticide use restrictions,Restricted,High,On Restricted Use Products (RUP) list,US EPA,https://www.epa.gov/pesticide-worker-safety/re...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [6]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:00:32.869514
